<a href="https://colab.research.google.com/github/Chachoummm/chess_engine/blob/main/train_chess_engineBon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Python Chess Engine Train

**libraries**

---

In [1]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

**connect to google drive**

---

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


**data**

---

import the chess games dataset from github

In [9]:
!git clone https://github.com/Chachoummm/chess_engine.git


Cloning into 'chess_engine'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 69 (delta 49), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [10]:
path_fischer = '/content/chess-games-dataset/Data/CSV_FISCHER'
path_morphy = '/content/chess-games-dataset/Data/CSV_MORPHY'
path_capablanca = '/content/chess_engine/Data/CSV_CAPABLANCA'
# ce serait intéressant de tester le modèle avec des datasets d'autres joueurs  

#files_fischer = glob.glob(path_fischer + "/*.csv")
#files_morphy = glob.glob(path_morphy + "/*.csv")
files_capablanca = glob.glob(path_capablanca + "/*.csv")
#index_with_nan = df.index[df.isnull().any(axis=1)]
            
li = []

for filename in files_capablanca:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

train = pd.concat(li, axis=0, ignore_index=True)
print (train)

      a1 b1    c1    d1    e1  ... to_e8 to_f8 to_g8 to_h8 good_move
0      R  N     B     Q     K  ...   0.0   0.0   0.0   0.0     False
1      R  N     B     Q     K  ...   0.0   0.0   0.0   0.0     False
2      R  N     B     Q     K  ...   0.0   0.0   0.0   0.0     False
3      R  N     B     Q     K  ...   0.0   0.0   0.0   0.0     False
4      R  N     B     Q     K  ...   0.0   0.0   0.0   0.0     False
...   .. ..   ...   ...   ...  ...   ...   ...   ...   ...       ...
53633  R  R  None  None  None  ...   0.0   0.0   0.0   0.0     False
53634  R  R  None  None  None  ...   0.0   0.0   0.0   0.0     False
53635  R  R  None  None  None  ...   0.0   0.0   0.0   0.0     False
53636  R  R  None  None  None  ...   0.0   0.0   0.0   0.0     False
53637  R  R  None  None  None  ...   0.0   0.0   0.0   0.0      True

[53638 rows x 193 columns]


In [11]:
train = shuffle(train)

In [12]:
train.shape

(53638, 193)

In [13]:
train.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,c2,d2,e2,f2,g2,h2,a3,b3,c3,d3,e3,f3,g3,h3,a4,b4,c4,d4,e4,f4,g4,h4,a5,b5,c5,d5,e5,f5,g5,h5,...,to_b4,to_c4,to_d4,to_e4,to_f4,to_g4,to_h4,to_a5,to_b5,to_c5,to_d5,to_e5,to_f5,to_g5,to_h5,to_a6,to_b6,to_c6,to_d6,to_e6,to_f6,to_g6,to_h6,to_a7,to_b7,to_c7,to_d7,to_e7,to_f7,to_g7,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
7861,R,N,B,Q,K,None,None,R,P,P,None,None,P,P,B,P,None,None,None,None,None,N,P,None,None,None,P,P,None,None,None,None,None,None,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
37066,None,None,None,None,None,None,None,None,None,None,None,None,None,P,None,K,None,None,None,None,None,None,P,None,P,None,p,R,None,None,None,P,Q,None,None,None,None,None,None,p,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4260,R,N,B,Q,K,B,None,R,P,P,P,P,None,P,P,P,None,None,None,None,None,N,None,None,None,None,None,None,P,None,None,None,None,None,None,None,p,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
5645,None,None,None,None,None,None,None,None,None,None,None,None,None,None,K,P,None,None,q,None,k,None,None,None,None,None,None,None,None,None,P,None,None,None,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
20458,None,None,None,None,None,None,None,None,None,None,None,None,K,P,None,None,None,None,None,None,None,N,P,P,None,b,None,None,None,None,None,None,None,None,None,P,None,None,None,p,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


**features**

---

In [14]:
features = list(train.iloc[:, 0:192].columns)

In [15]:
X = train[features]
y = train['good_move']
print (y)
y.dropna()
X.dropna()

7861     False
37066    False
4260     False
5645     False
20458    False
         ...  
31101    False
36384    False
50520    False
25519    False
33425    False
Name: good_move, Length: 53638, dtype: bool


,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,c2,d2,e2,f2,g2,h2,a3,b3,c3,d3,e3,f3,g3,h3,a4,b4,c4,d4,e4,f4,g4,h4,a5,b5,c5,d5,e5,f5,g5,h5,...,to_a4,to_b4,to_c4,to_d4,to_e4,to_f4,to_g4,to_h4,to_a5,to_b5,to_c5,to_d5,to_e5,to_f5,to_g5,to_h5,to_a6,to_b6,to_c6,to_d6,to_e6,to_f6,to_g6,to_h6,to_a7,to_b7,to_c7,to_d7,to_e7,to_f7,to_g7,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8
7861,R,N,B,Q,K,None,None,R,P,P,None,None,P,P,B,P,None,None,None,None,None,N,P,None,None,None,P,P,None,None,None,None,None,None,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37066,None,None,None,None,None,None,None,None,None,None,None,None,None,P,None,K,None,None,None,None,None,None,P,None,P,None,p,R,None,None,None,P,Q,None,None,None,None,None,None,p,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4260,R,N,B,Q,K,B,None,R,P,P,P,P,None,P,P,P,None,None,None,None,None,N,None,None,None,None,None,None,P,None,None,None,None,None,None,None,p,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5645,None,None,None,None,None,None,None,None,None,None,None,None,None,None,K,P,None,None,q,None,k,None,None,None,None,None,None,None,None,None,P,None,None,None,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20458,None,None,None,None,None,None,None,None,None,None,None,None,K,P,None,None,None,None,None,None,None,N,P,P,None,b,None,None,None,None,None,None,None,None,None,P,None,None,None,p,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31101,R,None,None,Q,None,R,K,None,None,B,None,None,N,None,P,P,P,P,N,B,None,P,None,None,None,None,None,P,None,None,None,None,None,None,P,p,None,None,None,n,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36384,None,R,None,R,None,None,K,None,None,None,None,None,None,P,None,P,P,None,Q,N,None,B,P,None,None,P,None,P,P,None,None,None,None,None,None,None,None,None,b,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50520,R,None,None,Q,None,None,None,R,None,None,None,None,None,P,K,P,P,None,None,B,B,N,P,None,N,None,None,P,None,None,None,None,p,P,None,p,P,p,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25519,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,P,None,None,None,None,P,None,K,p,None,None,None,None,p,None,P,None,None,None,None,None,r,P,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
  vocabulary = X[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

**input function**

---

In [17]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

**split data into batches**

---

In [18]:
def split_into_batches(df, batch_size=100000):
  nb_rows = len(df.index)
  intervals = []
  
  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)
  
  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)
  
  batches_X = []
  batches_y = []
  
  for i in range(0, len(intervals) - 1):
    batches_X.append(train.iloc[intervals[i]:intervals[i + 1], :][features])
    batches_y.append(train.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return batches_X, batches_y

In [19]:
batches_X, batches_y = split_into_batches(train)

**model**

---

In [26]:
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='/content/gdrive/MyDrive/latest_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/MyDrive/latest_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


**train model**

---

In [27]:
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
  input_functions.append(make_input_fn(df_X, df_y))

In [28]:
len(input_functions)



1

In [29]:
# train the model on all the input functions

i = 1
for input_function in input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)
  

# save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = '/content/gdrive/MyDrive/latest_model'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)

<======================================== NEW BATCH ========================================>
Batch: 1
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /content/gdrive/MyDrive/latest_model/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 21.2691
INFO:tensorflow:loss = 0.13044333, step = 100 (4.704 sec)
INFO:tensorflow:global_step/sec: 73.9151
INFO:tensorflow:loss = 0.14534082, step = 200 (1.353 sec)
INFO:tensorflow:global_step/sec: 71.5494
INFO:tensorflow:loss = 0.019966517, step = 300 (1.402 sec)
INFO:tensorflow:global_step/sec: 71.859
INFO:tensorflow:loss = 0.045302957, step = 400 (1.388 sec)
INFO:tensorflow:global_step/sec: 71.4252
INFO:tensorflow:loss = 0.01288794, step = 500 (1.402 sec)
INFO:tensorflo